In [1]:
import pandas as pd

In [2]:
market_data_path = "./data/test_prices_day_0.csv"
trade_history_path = "./data/test_trades_day_0.csv"

In [3]:
market_data = pd.read_csv(market_data_path, delimiter=";")

In [4]:
market_data.head()

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,0,0,AMETHYSTS,9998,1,9995.0,30.0,NaN,NaN,10005,30,NaN,NaN,NaN,NaN,10001.5,0.0
1,0,0,STARFRUIT,5036,30,NaN,NaN,NaN,NaN,5043,30,NaN,NaN,NaN,NaN,5039.5,0.0
2,0,100,AMETHYSTS,9996,1,9995.0,30.0,NaN,NaN,10002,6,10004.0,1.0,10005.0,30.0,9999.0,0.0
3,0,100,STARFRUIT,5041,4,5037.0,1.0,5036.0,30.0,5043,31,NaN,NaN,NaN,NaN,5042.0,0.0
4,0,200,AMETHYSTS,9996,1,9995.0,21.0,NaN,NaN,10004,1,10005.0,21.0,NaN,NaN,10000.0,0.0


In [5]:
trade_history = pd.read_csv(trade_history_path, delimiter=";")

In [6]:
trade_history.head(10)

,timestamp,buyer,seller,symbol,currency,price,quantity
0,100,NaN,NaN,STARFRUIT,SEASHELLS,5041.0,4
1,100,NaN,NaN,AMETHYSTS,SEASHELLS,10002.0,1
2,300,NaN,NaN,AMETHYSTS,SEASHELLS,9998.0,2
3,300,NaN,NaN,AMETHYSTS,SEASHELLS,9998.0,1
4,300,NaN,NaN,STARFRUIT,SEASHELLS,5037.0,6
5,500,NaN,NaN,STARFRUIT,SEASHELLS,5044.0,1
6,500,NaN,NaN,AMETHYSTS,SEASHELLS,10004.0,1
7,500,NaN,NaN,AMETHYSTS,SEASHELLS,10002.0,1
8,1300,NaN,NaN,STARFRUIT,SEASHELLS,5040.0,2
9,1400,NaN,NaN,STARFRUIT,SEASHELLS,5036.0,1


In [7]:
# Only focus on the first 300 timestamps
market_data = market_data.loc[lambda x: x['timestamp'] <= 300]
trade_history = trade_history.loc[lambda x: x['timestamp'] <= 300]

In [8]:
from run import Trader

In [9]:
from datamodel import Listing, Trade, OrderDepth, TradingState, Observation
from collections import defaultdict

class Backtest:
    """
        Simulate IMC exchange locally
    """
    def __init__(self, trader, listings, position_limits, market_data, trade_history, output_log):
        self.trader = trader
        self.listings = listings
        self.position_limits = position_limits
        self.market_data = market_data.sort_values(by='timestamp')
        # TODO (kyraz): I don't think we need to sort by symbol
        self.trade_history = trade_history.sort_values(by=['timestamp', 'symbol'])
        self.output_log = output_log

    def run(self):
        """
        Logic:
            - Iterate through market data timestamps, starting from market open t_0
            - For timestamp t_i, replay all market trades in trade history between t_{i-1} and t_i
            - For timestamp t_i, compute the markout pnl for each product at the current position
        """

        market_data_gp_ts = self.market_data.groupby('timestamp')
        trade_history_gp_ts = self.trade_history.groupby('timestamp')

        # Group trades by timestamp
        trades_by_timestamp = defaultdict()
        for timestamp, group in trade_history_gp_ts:
            trades = [
                Trade(
                    symbol=trade['symbol'], 
                    price=trade['price'],
                    quantity=trade['quantity'],
                    buyer=trade['buyer'],
                    seller=trade['seller'],
                    timestamp=trade['timestamp']
                ) for _, trade in group.iterrows()]
            trades_by_timestamp[timestamp] = trades

        for timestamp, group in market_data_gp_ts:
            market_trades = trades_by_timestamp[timestamp]


            # TODO: ADD, DELETE, EXECUTE orders
            # TODO: Construct order book
            # TODO: Construct trading state (positions)
            # TODO: Compute PnL›
        pass

    def _add_order(self):
        pass

    def _delete_order(self):
        pass

    def _execute_order(self, trades_to_execute):
        pass

    def calc_pnl(self):
        pass

In [41]:
def execute_buy_order(timestamp, order, order_depths, position, cash, position_limit, trades_by_timestamp, mid_prices, sandboxLog):
    trades = []
    order_depth = order_depths[order.symbol]
    mid_price = mid_prices[order.symbol]

    # Only cares about sell orders
    for price, volume in list(order_depth.sell_orders.items()):
        if price > order.price or order.quantity == 0:
            break

        trade_volume = min(abs(order.quantity), abs(volume))
        if abs(trade_volume + position[order.symbol]) <= int(position_limit[order.symbol]):
            trades.append(Trade(order.symbol, price, trade_volume, "SUBMISSION", "", timestamp))
            position[order.symbol] += trade_volume
            cash[order.symbol] -= price * trade_volume
            order_depth.sell_orders[price] += trade_volume
            order.quantity -= trade_volume
        else:
            print(f"Orders for product {order.symbol} exceeded limit of {position_limit[order.symbol]}")
            sandboxLog += f"\nOrders for product {order.symbol} exceeded limit of {position_limit[order.symbol]}"

        if order_depth.sell_orders[price] == 0:
            del order_depth.sell_orders[price]

    return trades, sandboxLog

def execute_sell_order(timestamp, order, order_depths, position, cash, position_limit, trades_by_timestamp, mid_prices, sandboxLog):
    trades = []
    order_depth = order_depths[order.symbol]
    mid_price = mid_prices[order.symbol]

    # Only cares about buy orders
    for price, volume in list(order_depth.buy_orders.items()):
        if price < order.price or order.quantity == 0:
            break

        trade_volume = min(abs(order.quantity), abs(volume))
        if abs(trade_volume + position[order.symbol]) <= int(position_limit[order.symbol]):
            trades.append(Trade(order.symbol, price, trade_volume, "SUBMISSION", "", timestamp))
            position[order.symbol] -= trade_volume
            cash[order.symbol] += price * trade_volume
            order_depth.buy_orders[price] -= trade_volume
            order.quantity += trade_volume
        else:
            print(f"Orders for product {order.symbol} exceeded limit of {position_limit[order.symbol]}")
            sandboxLog += f"\nOrders for product {order.symbol} exceeded limit of {position_limit[order.symbol]} set"

        if order_depth.buy_orders[price] == 0:
            del order_depth.buy_orders[price]

    return trades, sandboxLog

In [42]:
def execute_order(timestamp, orders, order_depths, position, cash, position_limit, trades_by_timestamp, mid_prices, sandboxLog):
    own_trades = []
    for symbol in symbols:
        orders_for_symbol = orders[symbol]
        for order in orders_for_symbol:
            if order.quantity > 0:
                # Execute buy order
                trades, sandboxLog = execute_buy_order(timestamp, order, order_depths, position, cash, position_limit, trades_by_timestamp, mid_prices, sandboxLog)
            else:
                # Execute sell order
                trades, sandboxLog = execute_sell_order(timestamp, order, order_depths, position, cash, position_limit, trades_by_timestamp, mid_prices, sandboxLog)
            own_trades += trades
    return own_trades, sandboxLog

In [56]:
def update_market_orders(timestamp, order_depths, trades_by_timestamp, mid_prices):
    # Modify market trade history
    trades_at_timestamp = trades_by_timestamp.get(timestamp, [])
    new_trades_at_timestamp = []
    for symbol in symbols:
        order_depth = order_depths[symbol]
        for trade in trades_at_timestamp:
            if symbol == trade.symbol:
                # Market trade conflicts with us, need to update the market trade
                if trade.price >= mid_price:
                    remain_quantity = abs(order_depth.sell_orders.get(trade.price, 0))
                else:
                    remain_quantity = abs(order_depth.buy_orders.get(trade.price, 0))
                if remain_quantity > 0:
                    new_quantity = min(remain_quantity, trade.quantity)
                    new_trades_at_timestamp.append(Trade(trade.symbol, trade.price, new_quantity, "", "", timestamp))
    trades_by_timestamp[timestamp] = new_trades_at_timestamp

In [57]:
listings = [
    Listing(symbol='AMETHYSTS', product='AMETHYSTS', denomination='SEASHELLS'),
    Listing(symbol='STARFRUIT', product='STARFRUIT', denomination='SEASHELLS'),
]

position_limit = {
    'AMETHYSTS': 20,
    'STARFRUIT': 20
}
trader = Trader()

In [63]:
import numpy as np
cash = {listing.symbol: 0 for listing in listings}
sandboxLog = []
current_position = {listing.symbol: 0 for listing in listings}
observations = None

#Debugging
market_data_gp_ts = market_data.groupby('timestamp')
trade_history_gp_ts = trade_history.groupby('timestamp')

# Group trades by timestamp
trades_by_timestamp = {}
for timestamp, group in trade_history_gp_ts:
    trades = [
        Trade(
            symbol=trade['symbol'], 
            price=trade['price'],
            quantity=trade['quantity'],
            buyer=trade['buyer'] if not np.isnan(trade['buyer']) else "",
            seller=trade['seller'] if not np.isnan(trade['buyer']) else "",
            timestamp=trade['timestamp']
        ) for _, trade in group.iterrows()]
    trades_by_timestamp[timestamp] = trades

In [64]:
LEVELS = 3
trader_data = ""
symbols = [listing.symbol for listing in listings]
# Own trades since last timestamp
own_trades = {listing.symbol: [] for listing in listings}
# Market trades since last timestamp
market_trades = {listing.symbol: [] for listing in listings}
mid_prices = {listing.symbol: 0. for listing in listings}
pnls = {listing.symbol: 0. for listing in listings}

for timestamp, group in market_data_gp_ts:
    order_depths = {}
    
    # Construct order book
    for _, row in group.iterrows():
        symbol = row['product']
        order_depth = OrderDepth()
        for i in range(1, LEVELS+1):
            # Bid
            bid_price = row[f'bid_price_{i}']
            bid_volume = row[f'bid_volume_{i}']
            if not np.isnan(bid_price) and not np.isnan(bid_volume):
                order_depth.buy_orders[int(bid_price)] = int(bid_volume)
                
            # Ask
            ask_price = row[f'ask_price_{i}']
            ask_volume = row[f'ask_volume_{i}']
            if not np.isnan(ask_price) and not np.isnan(ask_volume):
                order_depth.sell_orders[int(ask_price)] = -int(ask_volume)

            order_depths[symbol] = order_depth

            # Mid price
            if i == 1:
                mid_price = row['mid_price']
                mid_prices[symbol] = mid_price
    print(f"Trades by timestamp: {trades_by_timestamp.get(timestamp, [])}")
    print(f"Mid prices: {mid_prices}")

    # Assemble trading state
    trading_state = TradingState(trader_data, timestamp, listings, order_depths, own_trades, market_trades, current_position, observations)
    orders, conversions, trader_data = trader.run(trading_state)

    # Execute own orders, update market trade history
    own_trades, sandboxLog = execute_order(timestamp, orders, order_depths, current_position, cash, position_limit, trades_by_timestamp, mid_prices, sandboxLog)
    print(f"Own trades: {own_trades}")
    
    # Update market trades status
    update_market_orders(timestamp, order_depths, trades_by_timestamp, mid_prices)
    market_trades = {listing.symbol: [] for listing in listings}
    if timestamp in trades_by_timestamp.keys():
        for trade in trades_by_timestamp[timestamp]:
            market_trades[trade.symbol].append(trade)
    print(f"Market trades: {market_trades}")

    # Compute pnl
    for symbol in symbols:
        pnls[symbol] = cash[symbol] + mid_prices[symbol] * current_position[symbol]
        market_data
    print(f"Own position: {current_position}")
    print(f"PnL: {pnls}")

Trades by timestamp: []
Mid prices: {'AMETHYSTS': 10001.5, 'STARFRUIT': 5039.5}
AMETHYSTS, SELL, 9998, 1
STARFRUIT, SELL, 5036, 30
Orders for product STARFRUIT exceeded limit of 20
Own trades: [(AMETHYSTS, SUBMISSION << , 9998, 1, 0)]
Market trades: {'AMETHYSTS': [], 'STARFRUIT': []}
Own position: {'AMETHYSTS': -1, 'STARFRUIT': 0}
PnL: {'AMETHYSTS': -3.5, 'STARFRUIT': 0.0}
Trades by timestamp: [(STARFRUIT,  << , 5041.0, 4, 100), (AMETHYSTS,  << , 10002.0, 1, 100)]
Mid prices: {'AMETHYSTS': 9999.0, 'STARFRUIT': 5042.0}
AMETHYSTS, SELL, 9996, 1
STARFRUIT, SELL, 5041, 4
Own trades: [(AMETHYSTS, SUBMISSION << , 9996, 1, 100), (STARFRUIT, SUBMISSION << , 5041, 4, 100)]
Market trades: {'AMETHYSTS': [(AMETHYSTS,  << , 10002.0, 1, 100)], 'STARFRUIT': []}
Own position: {'AMETHYSTS': -2, 'STARFRUIT': -4}
PnL: {'AMETHYSTS': -4.0, 'STARFRUIT': -4.0}
Trades by timestamp: []
Mid prices: {'AMETHYSTS': 10000.0, 'STARFRUIT': 5040.0}
AMETHYSTS, SELL, 9996, 1
STARFRUIT, SELL, 5037, 1
Own trades: [(AMETHY

In [61]:
market_data.head()

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,0,0,AMETHYSTS,9998,1,9995.0,30.0,NaN,NaN,10005,30,NaN,NaN,NaN,NaN,10001.5,0.0
1,0,0,STARFRUIT,5036,30,NaN,NaN,NaN,NaN,5043,30,NaN,NaN,NaN,NaN,5039.5,0.0
2,0,100,AMETHYSTS,9996,1,9995.0,30.0,NaN,NaN,10002,6,10004.0,1.0,10005.0,30.0,9999.0,0.0
3,0,100,STARFRUIT,5041,4,5037.0,1.0,5036.0,30.0,5043,31,NaN,NaN,NaN,NaN,5042.0,0.0
4,0,200,AMETHYSTS,9996,1,9995.0,21.0,NaN,NaN,10004,1,10005.0,21.0,NaN,NaN,10000.0,0.0


In [ ]:
market_data.

In [287]:
import json
output = ""
output += "Sandbox logs:\n"
for i in sandboxLog:
    output += json.dumps(i, indent=2) + "\n"

        # output += "\n\n\n\nActivities log:\n"
        # market_data_csv = self.market_data.to_csv(index=False, sep=";")
        # market_data_csv = market_data_csv.replace("\r\n", "\n")
        # output += market_data_csv

        # output += "\n\n\n\nTrade History:\n"
        # output += json.dumps(self.trades, indent=2)

In [55]:
trades

[(AMETHYSTS,  << , 9998.0, 2, 300),
 (AMETHYSTS,  << , 9998.0, 1, 300),
 (STARFRUIT,  << , 5037.0, 6, 300)]

In [41]:
output_log = "backtest.log"
trader = Trader()
backtest = Backtest(trader, listings, position_limit, market_data, trade_history, output_log)
backtest.run()

KeyError: 0